In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline


# Building molecular Hamiltonians


.. meta::
    :property="og:description": Learn how to build electronic Hamiltonians of molecules.

    :property="og:image": https://pennylane.ai/qml/_images/water_structure.png

.. related::
   tutorial_vqe Variational quantum eigensolver

*Author: Alain Delgado. Last updated: 25 June 2022*

The ultimate goal of computational quantum chemistry is to unravel the
quantum effects that determine the structure and properties of molecules. Reaching
this goal is challenging since the characteristic energies associated with
these effects, e.g., the electronic correlation energy, are typically a tiny fraction
of the total energy of the molecule.

Accurate molecular properties can be computed from the wave function describing the
interacting electrons in a molecule. The **electronic** wave function
$\Psi(r)$ satisfies the `Schrödinger equation
<https://en.wikipedia.org/wiki/Schr%C3%B6dinger_equation>`_

\begin{align}H_e \Psi(r) = E \Psi(r),\end{align}

where $H_e$ and $E$ denote the electronic Hamiltonian and the
total energy of the molecule, respectively. When solving the latter equation,
the nuclei of the molecule can be treated as point particles whose coordinates
are fixed [#BornOpp1927]_. In this approximation, both the total energy and
the electronic Hamiltonian depend parametrically on the nuclear coordinates.


In this tutorial, you will learn how to use PennyLane to build a
representation of the electronic Hamiltonian $H_e$ that can be used to perform
**quantum** simulations of molecules [#yudong2019]_. First, we show how to define
the structure of the molecule in terms of the symbols and the coordinates of
the atoms. Next, we describe how to solve the `Hartree-Fock
equations <https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method>`_ for the target
molecule. Finally, we discuss some advanced features that can be used to simulate
more complicated systems.

Let's get started!

## Defining the molecular structure
In this example we construct the electronic Hamiltonian of the water molecule.


.. figure:: ../demonstrations/quantum_chemistry/water_structure.png
    :width: 30%
    :align: center

The structure of a molecule is defined by the symbols and the nuclear coordinates of
its constituent atoms. It can be specified using different `chemical file formats
<https://en.wikipedia.org/wiki/Chemical_file_format>`_. Within PennyLane, the molecular
structure is defined by providing a list with the atomic symbols and a one-dimensional
array with the nuclear coordinates in
`atomic units <https://en.wikipedia.org/wiki/Hartree_atomic_units>`_.


In [ ]:
import numpy as np

symbols = ["H", "O", "H"]
coordinates = np.array([-0.0399, -0.0038, 0.0, 1.5780, 0.8540, 0.0, 2.7909, -0.5159, 0.0])

The :func:`~.pennylane.qchem.read_structure` function can also be used to read the
molecular geometry from an external file.



In [ ]:
from pennylane import qchem

symbols, coordinates = qchem.read_structure("h2o.xyz")

The xyz format is supported.

## Solving the Hartree-Fock equations
The molecule's electronic Hamiltonian is commonly represented using the
`second-quantization <https://en.wikipedia.org/wiki/Second_quantization>`_ formalism,
which we will explore in more detail in the
next section. To that aim, a basis of **single-particle** states needs to be chosen.
In quantum chemistry these states are the
`molecular orbitals <https://en.wikipedia.org/wiki/Molecular_orbital>`_
which describe the wave function of a single electron in the molecule.

Molecular orbitals are typically represented as a linear combination of **atomic orbitals**.
The expansion coefficients in the atomic basis are calculated using the
`Hartree-Fock (HF) method <https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method>`_.
In the HF approximation, each electron in the molecule is treated as an **independent**
particle that moves under the influence of the Coulomb potential due to the nuclei, and a mean
field generated by all other electrons [#pople1977]_. The optimized coefficients are precisely
what we need to build the second-quantized Hamiltonian.

PennyLane provides a differentiable Hartree-Fock solver and the functionality to construct a
fully-differentiable molecular Hamiltonian.

## Building the Hamiltonian
In the second quantization formalism, the electronic wave function of the molecule
is represented in the occupation number basis. For $M$ *spin* molecular
orbitals, the elements of this basis are labelled as
$\vert n_0, n_1, \dots, n_{M-1} \rangle$, where $n_i = 0,1$
indicates the occupation of each orbital. In this representation, the electronic
Hamiltonian is given by

\begin{align}H = \sum_{p,q} h_{pq} c_p^\dagger c_q +
    \frac{1}{2} \sum_{p,q,r,s} h_{pqrs} c_p^\dagger c_q^\dagger c_r c_s,\end{align}

where $c^\dagger$ and $c$ are the electron creation
and annihilation operators, respectively, and the coefficients
$h_{pq}$ and $h_{pqrs}$ denote the one- and two-electron
Coulomb integrals [#ref_integrals]_ evaluated using the Hartree-Fock
orbitals.

We can use the states of $M$ qubits to encode any element
of the occupation number basis

\begin{align}\vert n_0, n_1, \dots, n_{M-1} \rangle \rightarrow \vert q_0 q_1 \cdots q_{M-1} \rangle.\end{align}

This implies that we need to map the fermionic operators onto operators
that act on qubits. This can be done by using the
`Jordan-Wigner <https://en.wikipedia.org/wiki/Jordan-Wigner_transformation>`_
transformation [#seeley2012]_ which allows us to decompose the fermionic Hamiltonian
into a linear combination of the tensor product of Pauli operators

\begin{align}H = \sum_j C_j \otimes_i \sigma_i^{(j)},\end{align}

where $C_j$ is a scalar coefficient and $\sigma_i$ represents an
element of the Pauli group $\{ I, X, Y, Z \}$.

In PennyLane we have the :func:`~.pennylane.qchem.molecular_hamiltonian`
function which encapsulates all the steps explained above. It simplifies the process of building
the electronic Hamiltonian to a single line of code. We just need to input the
symbols and the nuclear coordinates of the molecule, as shown below:



In [ ]:
H, qubits = qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits: {:}".format(qubits))
print("Qubit Hamiltonian")
print(H)

## Advanced features
The :func:`~.pennylane.qchem.molecular_hamiltonian` function allows us to define additional
keyword arguments to solve the Hartree-Fock equations of more complicated systems.
The net charge of the molecule may be specified to simulate positively or negatively
charged molecules. For a neutral system we choose



In [ ]:
charge = 0

We can also specify the
`spin multiplicity <https://en.wikipedia.org/wiki/Multiplicity_(chemistry)>`_. For the
water molecule, which contains ten electrons, the `Slater determinant
<https://en.wikipedia.org/wiki/Slater_determinant>`_ resulting from occupying the five
lowest-energy orbitals with two *paired* electrons per orbital has spin multiplicity one.
Alternatively, if we define an occupation where the first four orbitals are doubly occupied
and the next two are singly occupied by *unpaired* electrons, the HF state will have
multiplicity three.

|

.. figure:: ../demonstrations/quantum_chemistry/hf_references.png
    :width: 50%
    :align: center

|

For the neutral water molecule we have,



In [ ]:
multiplicity = 1

As mentioned above, molecular orbitals are represented as a linear combination
of atomic orbitals which are typically modeled as `Gaussian-type orbitals
<https://en.wikipedia.org/wiki/Gaussian_orbital>`_. We can specify different types
of `Gaussian atomic bases <https://www.basissetexchange.org/>`_. In this example we
choose a `minimal basis set
<https://en.wikipedia.org/wiki/Basis_set_(chemistry)#Minimal_basis_sets>`_.



In [ ]:
basis_set = "sto-3g"

PennyLane also allows us to define an active space [#truhlar2018]_ to perform quantum
simulations with a reduced number of qubits. This is done by classifying the molecular
orbitals as core, active, and external orbitals:

* Core orbitals are always occupied by two electrons.
* Active orbitals can be occupied by zero, one, or two electrons.
* The external orbitals are never occupied.

Within this approximation, a certain number of **active electrons** are allowed to
populate a finite set of **active orbitals**.

.. figure:: ../demonstrations/quantum_chemistry/sketch_active_space.png
    :width: 40%
    :align: center

<div class="alert alert-info"><h4>Note</h4><p>The number of active **spin-orbitals** determines the **number of qubits** required
    to perform the quantum simulations.</p></div>

For the water molecule in a minimal basis set we have a total of ten electrons
and seven molecular orbitals. In this example we define an symmetric active space with
four electrons and four active orbitals using
the :func:`~.pennylane.qchem.active_space` function:



In [ ]:
electrons = 10
orbitals = 7
core, active = qchem.active_space(electrons, orbitals, active_electrons=4, active_orbitals=4)

Viewing the results:



In [ ]:
print("List of core orbitals: {:}".format(core))
print("List of active orbitals: {:}".format(active))
print("Number of qubits: {:}".format(2 * len(active)))

Finally, we use the :func:`~.pennylane.qchem.molecular_hamiltonian` function to
build the resulting Hamiltonian of the water molecule:



In [ ]:
H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=charge,
    mult=multiplicity,
    basis=basis_set,
    active_electrons=4,
    active_orbitals=4,
)

print("Number of qubits required to perform quantum simulations: {:}".format(qubits))
print("Hamiltonian of the water molecule")
print(H)

In this case, since we have truncated the basis of molecular orbitals, the resulting
observable is an approximation of the Hamiltonian generated in the
section `hamiltonian`.

## OpenFermion-PySCF backend
The :func:`~.pennylane.qchem.molecular_hamiltonian` function can also be used to construct the
molecular Hamiltonian with a non-differentiable backend that uses the
`OpenFermion-PySCF <https://github.com/quantumlib/OpenFermion-PySCF>`_ plugin interfaced with the
electronic structure package `PySCF <https://github.com/sunqm/pyscf>`_. This
backend can be selected by setting ``method='pyscf'`` in
:func:`~.pennylane.qchem.molecular_hamiltonian`:



In [ ]:
H, qubits = qchem.molecular_hamiltonian(symbols, coordinates, method="pyscf")
print(H)

This backend requires the ``OpenFermion-PySCF`` plugin to be installed by the user with

.. code-block:: bash

   pip install openfermionpyscf

Additionally, if you have built your electronic Hamiltonian independently using
`OpenFermion <https://github.com/quantumlib/OpenFermion>`_ tools, it can
be readily converted to a PennyLane observable using the
:func:`~.pennylane.qchem.import_operator` function.

You have completed the tutorial! Now, select your favorite molecule and build its electronic
Hamiltonian.
To see how simple it is to implement the VQE algorithm to compute the ground-state energy of
your molecule using PennyLane, take a look at the tutorial :doc:`tutorial_vqe`.

## References

.. [#yudong2019]

    Yudong Cao, Jonathan Romero, *et al.*, "Quantum Chemistry in the Age of Quantum Computing".
    `Chem. Rev. 2019, 119, 19, 10856-10915.
    <https://pubs.acs.org/doi/10.1021/acs.chemrev.8b00803>`_

.. [#BornOpp1927]

    M. Born, J.R. Oppenheimer, "Quantum Theory of the Molecules".
    `Annalen der Physik 84, 457-484 (1927)
    <https://onlinelibrary.wiley.com/doi/abs/10.1002/andp.19273892002>`_

.. [#pople1977]

    Rolf Seeger, John Pople. "Self‐consistent molecular orbital methods. XVIII. Constraints and
    stability in Hartree–Fock theory". `Journal of Chemical Physics 66,
    3045 (1977). <https://aip.scitation.org/doi/abs/10.1063/1.434318>`_

.. [#ref_integrals]

    J.T. Fermann, E.F. Valeev, "Fundamentals of Molecular Integrals Evaluation".
    `arXiv:2007.12057 <https://arxiv.org/abs/2007.12057>`_

.. [#seeley2012]

    Jacob T. Seeley, Martin J. Richard, Peter J. Love. "The Bravyi-Kitaev transformation for
    quantum computation of electronic structure". `Journal of Chemical Physics 137, 224109 (2012).
    <https://aip.scitation.org/doi/abs/10.1063/1.4768229>`_

.. [#truhlar2018]

    J.J. Bao, S.S. Dong, L. Gagliardi, D.G. Truhlar. "Automatic Selection of an
    Active Space for Calculating Electronic Excitation Spectra by MS-CASPT2 or MC-PDFT".
    `Journal of Chemical Theory and Computation 14, 2017 (2018).
    <https://pubs.acs.org/doi/abs/10.1021/acs.jctc.8b00032>`_

## About the author



.. bio:: Alain Delgado
   :photo: ../_static/alain-delgado.png

   Alain Delgado Gran is a senior quantum scientist working at Xanadu. He has a background in quantum many-body systems and electronic structure methods. His work focuses on the application of quantum algorithms for simulating advanced materials.

